In [143]:
from bs4 import BeautifulSoup
import requests
import re
import googlemaps
import csv
import json

In [163]:
gmaps = googlemaps.Client(key=gmapKey)

In [294]:
mfaId = '519903'
itemId = 'mfa_'+mfaId
tags = ['african-descent']

In [295]:
website = requests.get('https://collections.mfa.org/objects/'+mfaId)
print(website)

<Response [200]>


In [296]:
soup = BeautifulSoup(website.content, 'html.parser')

In [297]:
print(soup.prettify())

<!DOCTYPE html>
<html data-locale="en" lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="Apache Tapestry Framework (version 5.4.3)" name="generator"/>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&amp;l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-GWJF');
  </script>
  <!-- End Google Tag Manager -->
  <title>
   Vase – Works – Museum of Fine Arts, Boston
  </title>
  <link href="/assets/meta/zf584fe3/css/bootstrap.min.css" media="screen" rel="stylesheet" type="text/css"/>
  <!-- Favicon for Internal Skin -->
  <link 

In [298]:
provhtml = soup.find_all("div", {"class":"detailField provenanceField"})

In [299]:
dischtml = soup.find_all("div", {"class":"detailField webDescriptionField"})

In [300]:
if len(provhtml) >= 2:
    raise ValueError("multiple fields detected")

In [301]:
provStr = provhtml[0].contents[1].text # Contents[0] is the title, [1] should be the data 

In [302]:
provStr = provStr.split('NOTE:')[0].split('\r')[0]

In [303]:
provStr

'By 1990, owned by American Decorative Arts (gallery), Northampton, Massachusetts; January 24, 1990, sold by Chris Kennedy of American Decorative Arts to John Axelrod, Boston, Massachusetts; 2008, promised gift of John Axelrod to the Museum; 2014, gift of Axelrod to the MFA. (Accession date: October 29, 2014)'

In [304]:
provLst = re.split(' to ',provStr)

In [307]:
provLst

['By 1990, owned by American Decorative Arts (gallery), Northampton, Massachusetts; January 24, 1990, sold by Chris Kennedy of American Decorative Arts',
 'John Axelrod, Boston, Massachusetts; 2008, promised gift of John Axelrod',
 'the Museum; 2014, gift of Axelrod',
 'the MFA. (Accession date: October 29, 2014)']

In [306]:
times = []
places = []
addresses = []
coordinates = []
for element in provLst:
    years = re.findall(r"[0-9]{4}", element)
    if not years:
        raise ValueError("year missing")
    times.append(years[-1])
    place = list(dict.fromkeys([re.sub(r'[^\w\s]','',word) for word in element.split() if word[0].isupper() ]))
    place = ' '.join(place)
    places.append(place)
    geocode_result = gmaps.geocode(place)
    addresses.append(re.sub(r',',';',geocode_result[0]['formatted_address']))
    ECo,NCo = geocode_result[0]['geometry']['location'].values()
    coordinates.append((ECo,NCo))

In [308]:
with open('../provenance/'+itemId+'.csv', mode='w') as employee_file:
    prov_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for ii,(time,place,address,coordinate) in enumerate(zip(times,places,addresses,coordinates)):
        prov_writer.writerow([ii,time,place,address,coordinate[0],coordinate[1]])

In [309]:
internalHttp = soup.find_all("img", {"class":"disable-click"})[0]['src']
image_url = 'https://collections.mfa.org/' + internalHttp

In [310]:
try: 
    artist = soup.find_all("div", {"class":"detailField peopleField"})[0].text
except IndexError:
    artist = 'Unknown'

In [311]:
metadata = {'disc': dischtml[0].text, \
            'name' : soup.find_all("h2")[0].text, \
            'by': artist, \
            'image_url': image_url,
            'yearmade' : re.findall(r"[0-9]{4}",soup.find_all("div", {"class":"detailField displayDateField"})[0].text)[-1],
            'tags':tags}

In [312]:
with open('../metadata/'+itemId+'.json', 'w') as outfile:  
    json.dump(metadata, outfile) 

In [292]:
'''
internalHttp = soup.find_all("img", {"class":"disable-click"})[0]['src']
image_url = 'https://collections.mfa.org/' + internalHttp
r = requests.get(image_url, stream = True)
r.raw.decode_content = True
with open('../media/'+itemId+'.jpg', 'wb') as handler:
    handler.write(r.raw.read())
'''

'\ninternalHttp = soup.find_all("img", {"class":"disable-click"})[0][\'src\']\nimage_url = \'https://collections.mfa.org/\' + internalHttp\nr = requests.get(image_url, stream = True)\nr.raw.decode_content = True\nwith open(\'../media/\'+itemId+\'.jpg\', \'wb\') as handler:\n    handler.write(r.raw.read())\n'